In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3 # backbone model
from tensorflow.keras.layers import BatchNormalization , Dense , Dropout , GlobalAveragePooling2D

In [ ]:
# Create backbone model   
def createInceptionV3(input_shape):
    model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    pooling=None)
    return model

basemodel = createInceptionV3((105, 105, 3))
basemodel.trainable = False

def create_model(base_model):
    
    x =  BatchNormalization()(base_model.output)
    x =  GlobalAveragePooling2D()(x)
    
    # Font Family
    dense_family = Dense(1024,activation = 'relu')(x)
    dropout_family = Dropout(0.2)(dense_family)
    output_family = Dense(10,activation = 'softmax')(dropout_family)
 
    model = tf.keras.Model(inputs = base_model.input , outputs = output_family)
    
    return model

model = create_model(basemodel)

In [ ]:
# Check for input path!
#model.load_weights('../input/font-regcog-inception/inception_best.h5')
model.load_weights('../input/outputfontreg-1/inception_best.h5')

In [ ]:
model.compile(optimizer =  tf.keras.optimizers.Adam(),
              # กำหนด loss แบบ list ตาม order ที่create model or use 1 for all
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = ['acc']
            )

In [ ]:
id_test = pd.read_csv('../input/final-test-image/label_test.csv')

In [ ]:
id_test['id'] = id_test['id'] + '.png'

In [ ]:
id_test.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
test_datagen = ImageDataGenerator(preprocessing_function = tf.keras.applications.inception_v3.preprocess_input)


test_generator = test_datagen.flow_from_dataframe(
     dataframe= id_test[:], # [:] for full data
     directory="../input/final-test-image/image_test",
     target_size=(105, 105),
     x_col = "id",
     shuffle=False,
     seed=69,
     batch_size=128,
     class_mode = None)

In [ ]:

# Test image in batch
import matplotlib.pyplot as plt
import numpy as np
img = np.squeeze(next(iter(test_generator))[0])
plt.imshow(img)


In [ ]:
pred = model.predict(test_generator)

In [ ]:
pred.shape

In [ ]:
metadata_aux = pd.read_csv('../input/font-detection-dataset-crop/label_train.csv')
# map Fontfamily to interger
unique_vals = metadata_aux['fontFamily'].unique()
metadata_aux['fontFamily_int'] = metadata_aux['fontFamily'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

In [ ]:
metadata_aux.head()

In [ ]:
#metadata_aux[metadata_aux['fontStyle_int'] == 0]['fontStyle'].sample().to_string().split(' ')[-1]

In [ ]:
def map_interger_to_label(mode = None, integer = None):
    if mode == 'family':
        return metadata_aux[metadata_aux['fontFamily_int'] == integer]['fontFamily'].sample().to_string().split(' ')[-1]
    elif mode == 'weight':
        return metadata_aux[metadata_aux['fontWeight_int'] == integer]['fontWeight'].sample().to_string().split(' ')[-1]
    elif mode == 'style':
        return metadata_aux[metadata_aux['fontStyle_int'] == integer]['fontStyle'].sample().to_string().split(' ')[-1]
    elif mode == 'size':
        return metadata_aux[metadata_aux['fontSize_int'] == integer]['fontSize'].sample().to_string().split(' ')[-1]

In [ ]:
from tqdm.notebook import tqdm as tqdm

In [ ]:
pred.shape

In [ ]:
test_label = {}
for i in tqdm(range(len(id_test))):
    prediction = None
    parentId = id_test.loc[i].to_string().split(' ')[-1].split('.')[0]
    # Get font family prediction
    prediction = np.argmax(pred[i])
    prediction = map_interger_to_label(mode = 'family', integer = prediction)
    test_label[parentId] =  prediction
    

In [ ]:
len(test_label)

In [ ]:
sample_submission_df = pd.read_csv('../input/finaltestsubmit/sample_submission.csv')
sample_submission_df.set_index('Id',inplace=True)

In [ ]:
# map output to submission.csv
for idx , label in tqdm(test_label.items()):
    doc_id = idx
    family = label
    #style = label['Style']
    #weight = label['Weight']
    #size = label['Size']
    
    
    sample_submission_df.loc[f'{idx}_name'] = family
    sample_submission_df.loc[f'{idx}_file'] = f'{family}.ttf'.replace('_',' ')
    #sample_submission_df.loc[f'{idx}_size'] = size
    #sample_submission_df.loc[f'{idx}_style-weight'] = f'{style}-{weight}'


In [ ]:
len(test_label)

In [ ]:
sample_submission_df.to_csv('./sample_submission.csv')